In [2]:
import geopandas as gpd
import geoai


In [3]:
satellite_imagery = './pabbi_satellite_image_COG.tif'
vector_file = './pabb_crop_V3.geojson'

In [13]:

gdf = gpd.read_file(vector_file)

print(f"Total Number of Parcels {len(gdf)}")

print(f"Number of Parcels with values (other than null): {gdf['Landuse_Ma'].notna().sum()}")

print(f"Details of Crop are : {gdf['Landuse_Ma'].value_counts()}")

Total Number of Parcels 7012
Number of Parcels with values (other than null): 6515
Details of Crop are : Landuse_Ma
Agriculture     5759
Stream           302
Other            229
Road/Streets     172
Graveyard         29
Built up          24
Name: count, dtype: int64


# Setting Class Ids for Land use Before generating Raster

In [14]:
label_map = {
    'Agriculture': 1,
    'Stream': 2,
    'Other': 3,
    'Road/Streets': 4,
    'Graveyard': 5,
    'Built up': 6
}

gdf['class_id'] = gdf['Landuse_Ma'].map(label_map)

gdf = gdf[gdf['class_id'].notna()]
gdf['class_id'] = gdf['class_id'].astype('uint8')
gdf=gdf.to_crs('EPSG:4326')
print(gdf.crs)

EPSG:4326


In [15]:
print(gdf['class_id'].unique())
print(gdf['class_id'].isnull().sum())
print(gdf['class_id'].dtype)


[1 6 2 4 5 3]
0
uint8


In [13]:
# Writing to temporary file
gdf.to_file('temp_cleaned_vector.geojson')

This code didnot work so i have to use the above code for manual vector to raster conversion.
geoai.vector_to_raster(
    vector_path=vector_file,
    reference_raster=satellite_imagery,
    attribute_field='class_id',
    output_path='Labelraster_For_Unet_v4.tif',
    dtype='uint8',
    plot_result=True,
    nodata=0
    )

In [6]:
geoai.view_raster('Labelraster_Manual.tif', cmap='tab20')

# Unet Part to create image tiles / patches

In [5]:
vector_path = 'temp_cleaned_vector.geojson'
labeled_raster_path = 'labelraster_Manual.tif'
out_folder = 'landcover_pabbi_satellite'

In [22]:

tiles = geoai.export_geotiff_tiles(
    in_raster = satellite_imagery,
    out_folder= out_folder,
    in_class_data= labeled_raster_path,
    #class_value_field= 'labels',
    tile_size=512,
    stride=256,
    buffer_radius=0,
    skip_empty_tiles=True,
)

Detected in_class_data as raster: labelraster_Manual.tif
Raster CRS: EPSG:4326
Raster dimensions: 31447 x 23080

Raster info for ./pabbi_satellite_image_COG.tif:
  CRS: EPSG:4326
  Dimensions: 31447 x 23080
  Resolution: (2.6822089999998085e-06, 2.682208999999812e-06)
  Bands: 3
  Bounds: BoundingBox(left=71.731650084257, bottom=33.986464888201006, right=71.81599751067999, top=34.048370271921)
Found 6 unique classes in raster: [1 2 3 4 5 6]


Generated: 4477, With features: 4477: 100%|██████████| 10980/10980 [03:04<00:00, 59.53it/s] 


------- Export Summary -------
Total tiles exported: 4477
Tiles with features: 4477 (100.0%)
Average feature pixels per tile: 440491.3
Output saved to: landcover_pabbi_satellite

------- Georeference Verification -------


In [ ]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=4,
    num_classes=13,
    batch_size=8,
    num_epochs=50,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
    plot_curves=True,
)

Using device: cuda
Found 4477 image files and 4477 label files
Training on 3581 images, validating on 896 images
Starting training with unet + resnet34
Model parameters: 24,441,245
Epoch: 0, Batch: 0/896, Loss: 2.5749, Time: 30.16s
Epoch: 0, Batch: 10/896, Loss: 1.7424, Time: 3.97s
Epoch: 0, Batch: 20/896, Loss: 1.6876, Time: 4.00s
Epoch: 0, Batch: 30/896, Loss: 1.3818, Time: 3.96s
Epoch: 0, Batch: 40/896, Loss: 1.7097, Time: 3.98s
Epoch: 0, Batch: 50/896, Loss: 0.7771, Time: 3.98s
Epoch: 0, Batch: 60/896, Loss: 0.8404, Time: 3.98s
Epoch: 0, Batch: 70/896, Loss: 0.8106, Time: 4.00s
Epoch: 0, Batch: 80/896, Loss: 0.2984, Time: 3.98s
Epoch: 0, Batch: 90/896, Loss: 1.3713, Time: 4.01s
Epoch: 0, Batch: 100/896, Loss: 0.3584, Time: 4.01s
Epoch: 0, Batch: 110/896, Loss: 1.1237, Time: 4.00s
Epoch: 0, Batch: 120/896, Loss: 0.8955, Time: 4.00s
Epoch: 0, Batch: 130/896, Loss: 1.3193, Time: 4.03s
Epoch: 0, Batch: 140/896, Loss: 1.3070, Time: 4.01s
Epoch: 0, Batch: 150/896, Loss: 0.6727, Time: 3.9

In [ ]:
# Define paths
masks_path = "naip_test_semantic_prediction.tif"
model_path = f"{out_folder}/unet_models/best_model.pth"

In [ ]:
# Run semantic segmentation inference
geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=4,
    num_classes=13,
    window_size=512,
    overlap=256,
    batch_size=4,
)

In [7]:
geoai.write_colormap(masks_path, train_landcover_path, output=masks_path)

NameError: name 'masks_path' is not defined

In [ ]:
geoai.view_raster(masks_path, basemap=test_raster_url)